# 멀티모달 LLM “gemma3-27b”로 텍스트·이미지 분석

- Visual & Text Processing·다국어 번역·요약 지원
- API 무료 분당 30회, 하루 15,000회까지 사용 가능
- 분석할 때 최소 time.sleep(2) 필요

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 패키지 설치와 불러오기

In [ ]:
!pip install --quiet google-generativeai pillow requests python-dotenv

In [ ]:
import os
import requests, time, json, re
from io import BytesIO  # 파일이 없어도, 메모리 속 데이터(바이트열) 를 파일처럼 다룰 수 있게 해줌. 웹에서 받은 이미지/파일을 저장하지 않고 바로 사용
from PIL import Image
from IPython.display import display
from google import genai
import pandas as pd

## 구글 gemma-3-27b 사용 인증
- 보안비밀에 API Key 저장

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

## 이미지 로드 함수 (URL/로컬 파일 모두 지원, 재시도 포함)

In [ ]:
import requests
import time
from PIL import Image
from io import BytesIO

RETRY_LIMIT = 3

def load_image(source: str) -> Image.Image:
    """
    다양한 형식의 이미지를 로드하는 함수
    JPG, PNG, WEBP, GIF 등 모든 형식 지원
    """
    # HTTP 요청용 세션 생성
    session = requests.Session()
    # 웹사이트 차단 방지용 User-Agent 설정
    session.headers.update({'User-Agent': 'Mozilla/5.0'})

    # 재시도 루프
    for attempt in range(1, RETRY_LIMIT + 1):
        try:
            # 웹 URL인지 로컬 파일인지 확인
            if source.startswith(("http://", "https://")):
                # 웹에서 이미지 다운로드
                resp = session.get(source, timeout=10)
                resp.raise_for_status()  # HTTP 에러 확인
                img = Image.open(BytesIO(resp.content)) # 메모리에 올린 뒤 이미지 객체로 열기
            else:
                # 로컬 파일 열기
                img = Image.open(source)

            # 투명도가 있는 이미지 처리 (WEBP, PNG 등)
            if img.mode in ('RGBA', 'LA', 'P'):
                # 흰색 배경 이미지 생성
                background = Image.new('RGB', img.size, (255, 255, 255))
                # 팔레트 모드면 RGBA로 먼저 변환
                if img.mode == 'P':
                    img = img.convert('RGBA')
                # 투명도 마스크가 있으면 적용
                if len(img.split()) > 3:
                    background.paste(img, mask=img.split()[-1])
                else:
                    background.paste(img)
                img = background
            else:
                # 일반 이미지는 RGB로 변환
                img = img.convert("RGB")

            return img

        except Exception as e:
            print(f"[경고] 이미지 로드 실패 {attempt}/{RETRY_LIMIT}: {e}")
            # 마지막 시도가 아니면 1초 대기
            if attempt < RETRY_LIMIT:
                time.sleep(1)

    # 모든 시도 실패시 예외 발생
    raise RuntimeError(f"이미지를 불러올 수 없습니다: {source}")

## 추론 함수 (텍스트 이미지 모두 가능)

In [ ]:
from google import genai
import os
import re
import json
import time
from IPython.display import display

# 이전 분석 맥락 초기화용 프롬프트
RESET_PROMPT = "이전 대화는 모두 무시하고, 아래 지시에 따라서만 응답해 주세요.\n\n"

def infer(source: str, prompt: str) -> dict:
    """
    멀티모달 AI 추론 함수
    - 이미지(URL/로컬 파일/폴더) → 이미지+텍스트 분석
    - 순수 텍스트 → 텍스트 전용 분석
    - 항상 새로운 분석으로 시작 (이전 맥락 무시)
    """
    try:
        # 이미지 소스인지 판별 (URL, 파일, 폴더 경로 모두 지원)
        is_image_source = (
            source.startswith(("http://", "https://")) or  # 웹 URL
            os.path.isfile(source) or                      # 파일 경로
            (os.path.isdir(os.path.dirname(source)) and    # 폴더 내 파일
             any(source.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.webp', '.gif', '.bmp']))
        )

        # 콘텐츠 준비
        if is_image_source:
            # 이미지 모드: 이미지 로드 및 표시
            img = load_image(source)
            display(img)  # Jupyter/Colab에서 이미지 미리보기
            print(f"📷 이미지 로드 완료: {img.size}")

            # 이미지와 프롬프트를 함께 전달
            contents = [RESET_PROMPT + prompt, img]
        else:
            # 텍스트 모드: 텍스트 내용을 프롬프트에 결합
            text_payload = RESET_PROMPT + prompt.strip() + "\n\n" + source.strip()
            contents = [text_payload]

        # AI 모델 호출
        print("🤖 AI 분석 중...")
        client = genai.Client(api_key=GOOGLE_API_KEY)
        resp = client.models.generate_content(
            model="gemma-3-27b-it",
            contents=contents
        )

        # API 호출 제한 대응 (무료 계정: 분당 30회)
        time.sleep(1)

        # 응답 텍스트 추출
        raw = resp.text.strip()
        print("📝 원문 응답:")
        print(raw)

        # JSON 형태 응답 추출 시도
        # 1) 코드 블록 내 JSON 찾기
        pattern = r'```(?:json)?\s*(\{[\s\S]*?\})\s*```|(\{[\s\S]*?\})' # ... 블록 안에 있는 JSON (```json { ... } ``` 형식) 또는 그냥 { ... } 로 둘러싸인 JSON 객체
        for group1, group2 in re.findall(pattern, raw):
            candidate = (group1 or group2).strip()
            try:
                parsed_json = json.loads(candidate)
                print("✅ JSON 파싱 성공")
                return parsed_json
            except json.JSONDecodeError:
                continue

        # 2) 코드 블록 제거 후 전체 파싱 시도
        cleaned_text = re.sub(r'```json|```', '', raw).strip()
        try:
            parsed_json = json.loads(cleaned_text)
            print("✅ 정리된 텍스트에서 JSON 파싱 성공")
            return parsed_json
        except json.JSONDecodeError:
            pass

        # 3) JSON 파싱 실패시 원문 텍스트 반환
        print("⚠️ JSON 파싱 실패, 원문 텍스트 반환")
        return {"text": raw}

    except Exception as e:
        # 모든 예외 처리
        print(f"❌ 오류 발생: {e}")
        return {"error": str(e)}

## 텍스트 분석: 프롬프트, 데이터, 분석

In [ ]:
# 프롬프트
TEXT_PROMPT = """
당신은 기사 속 정보원 분석 전문가입니다.
다음 기사에서 정보원을 추출하고, 정보원이 묘사된 프레임을 긍정/중립/부정으로 판단해, JSON으로 반환해주세요."
#결과 예시:
{"sources": [정보원A, 정보원B], "frames": [중립, 부정]}
기사:
"""

# 분석 대상 텍스트
df = pd.DataFrame({
    "id": [0,1],
    "articles": [
    "Donald Trump와 Nancy Pelosi가 회의장에서 격렬히 논쟁을 벌이며 서로의 정책을 비판했다.",
    "경제 전문가 A는 한국 경제가 올해 빠른 회복세를 보일 것이라고 예측했고, 전문가 B는 글로벌 불확실성 때문에 경제 회복이 더딜 것이라고 평가했다."
]
})

# 분석
results = []
for id, article in zip(df["id"], df["articles"]):
    print(f"\n=== 기사 {id} 분석 시작 ===")
    try:
        result = infer(source=article, prompt=TEXT_PROMPT)
        result["id"] = id
    except Exception as e:
        result = {
            "sources": [],       # 분석 실패 시 빈 리스트
            "frames": [],        # 분석 실패 시 빈 딕셔너리
            "id": id,
        }
    print("분석 결과:", result)
    results.append(result)
    time.sleep(1)  # API 무료 분당 30회

# 결과 정리
df = pd.DataFrame(results)

## 이미지 분석: 프롬프트, 데이터, 분석

In [ ]:
# 프롬프트
IMAGE_PROMPT = '''
당신은 보도사진 분석 전문가입니다. 제공된 사진에서 Donald Trump를 찾아 그의 감정과 역동성을 분석하세요.

[분석 절차]
1. 먼저 사진에서 Donald Trump 식별
2. Trump가 발견되면 다음 기준으로 분석

[분석 기준]
1. 감정 분석 (얼굴 표정 기준):
   - positive: 미소, 웃음, 밝은 표정, 눈가 주름 등 긍정적 표현
   - negative: 찡그림, 눈썹 찌푸림, 입꼬리 하향, 경직된 표정
   - neutral: 무표정, 평상시 표정, 뚜렷한 감정 표현 없음

2. 역동성 평가 (신체 움직임 기준):
   - high: 양팔을 크게 벌리거나 몸통 회전, 강렬한 제스처
   - medium: 한 손 제스처, 자연스러운 걸음, 일반적 동작
   - low: 정적 자세, 최소한의 움직임, 제한된 동작

[출력 형식]
JSON: {
  "trump_present": true/false,
  "emotion": "positive/negative/neutral" (Trump가 있을 경우만),
  "dynamism": "high/medium/low" (Trump가 있을 경우만),
  "analysis": "간단한 분석 근거"
}
Trump가 사진에 없으면 trump_present: false만 반환하세요. 출력 형식 이외에 추가 설명을 하지 말 것.
제공된 이미지를 분석해주세요.'''

# 분석 대상 이미지
images = [
    "https://www.minnpost.com/wp-content/uploads/2018/09/donald-trump-high-angle_main.jpg",
    "https://ichef.bbci.co.uk/ace/ws/640/cpsprodpb/60f7/live/f19e4620-8258-11f0-a34f-318be3fb0481.jpg.webp",
    "https://www.chosun.com/resizer/v2/QHMG56DYYNO6JGSNIWMF4QQCCI.jpg?auth=061c859d87af03768c186d560e6ddcda48feab7408e293bbadc928ff15704945&width=616",
    "https://file2.nocutnews.co.kr/newsroom/image/2019/02/28/20190228054724516754_6_710_359.jpg",
    "/content/drive/MyDrive/llm_내용분석/images/test/트럼프1.jpg"
]

# 분석
results_image=[]
for image in images[:]:
    try:
        result_image = infer(source=image,  prompt=IMAGE_PROMPT)
        print(f"=== {image} ===\n{result_image}\n")
    except Exception as e:
        print(f"[ERROR] {image} 처리 실패: {e}")
    result_image['image_src'] =  image
    results_image.append(result_image)
    time.sleep(1) # API 무료 분당 30회

# 결과 정리
df_result=pd.DataFrame(results_image)

# 앱 만들기: streamlit 활용

Streamlit은 Python으로 웹 애플리케이션을 쉽고 빠르게 만들 수 있는 오픈소스 프레임워크

웹 개발 지식 없이도 인터랙티브한 대시보드와 앱 개발

특징
- 간단한 문법: Python 스크립트처럼 작성하면 자동으로 웹앱 생성
- 실시간 업데이트: 코드 수정시 자동으로 브라우저 새로고침
- 다양한 위젯: 슬라이더, 버튼, 차트 등 풍부한 UI 컴포넌트
- 데이터 시각화: Matplotlib, Plotly, Altair 등 차트 라이브러리 지원
- 무료 배포: Streamlit Cloud로 무료 호스팅

공식 사이트
- 공식 홈페이지: https://streamlit.io
- 공식 문서: https://docs.streamlit.io

학습 자료
- 공식 튜토리얼: https://docs.streamlit.io/get-started/tutorials
- 30 Days of Streamlit: https://30days.streamlit.app
- Streamlit Cookbook: https://docs.streamlit.io/knowledge-base/tutorials

예제 및 갤러리
- 앱 갤러리: https://streamlit.io/gallery
- Community Gallery: https://streamlit.io/gallery?category=featured
- GitHub 예제: https://github.com/streamlit/streamlit/tree/develop/examples

배포 플랫폼
- Streamlit Cloud: https://share.streamlit.io (무료)
- Hugging Face Spaces: https://huggingface.co/spaces
- Heroku: https://heroku.com
- Railway: https://railway.app

In [ ]:
# app.py
import os, io, time, json, re, requests
from io import BytesIO   # 파일이 없어도, 메모리 속 데이터(바이트열) 를 파일처럼 다룰 수 있게 해줌. 웹에서 받은 이미지/파일을 저장하지 않고 바로 사용
from PIL import Image
import streamlit as st
from google import genai

st.set_page_config(page_title="멀티모달 LLM 데모 (Gemma 3 27B)", page_icon="🤖", layout="wide")

# ===== Sidebar: 설정 =====
st.sidebar.title("🔧 설정")
api_key = st.sidebar.text_input("GOOGLE_API_KEY", value=os.getenv("GOOGLE_API_KEY", ""), type="password")
model_name = st.sidebar.selectbox("모델", ["gemma-3-27b-it"], index=0)
rate_delay = st.sidebar.number_input("호출간 대기(초)", value=1.0, step=0.5, min_value=0.0)

if not api_key:
    st.warning("사이드바에 GOOGLE_API_KEY를 입력하거나 환경변수로 설정해 주세요.")
    st.stop()

client = genai.Client(api_key=api_key)

# ===== 유틸 =====
RETRY_LIMIT = 3

def load_image(source: str) -> Image.Image:
    """URL/로컬 모두 지원 + 투명 배경 보정."""
    session = requests.Session()
    session.headers.update({"User-Agent": "Mozilla/5.0"})
    last_err = None
    for _ in range(RETRY_LIMIT):
        try:
            if source.startswith(("http://", "https://")):
                resp = session.get(source, timeout=10)
                resp.raise_for_status()
                img = Image.open(BytesIO(resp.content))
            else:
                img = Image.open(source)

            if img.mode in ("RGBA", "LA", "P"):
                bg = Image.new("RGB", img.size, (255, 255, 255))
                img = img.convert("RGBA")
                bg.paste(img, mask=img.split()[-1] if len(img.split()) > 3 else None)
                img = bg
            else:
                img = img.convert("RGB")
            return img
        except Exception as e:
            last_err = e
            time.sleep(0.5)
    raise RuntimeError(f"이미지 로드 실패: {last_err}")

RESET_PROMPT = "이전 대화는 무시하고, 아래 지시에만 응답하세요.\n\n"

def try_parse_json(raw_text: str):
    """응답에서 JSON 추출 시도."""
    pattern = r'```(?:json)?\s*(\{[\s\S]*?\})\s*```|(\{[\s\S]*?\})'
    for g1, g2 in re.findall(pattern, raw_text):
        cand = (g1 or g2).strip()
        try:
            return json.loads(cand)
        except json.JSONDecodeError:
            pass
    cleaned = re.sub(r'```json|```', '', raw_text).strip()
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        return None

def infer_text(article: str, prompt: str):
    contents = [RESET_PROMPT + prompt.strip() + "\n\n" + article.strip()]
    resp = client.models.generate_content(model=model_name, contents=contents)
    time.sleep(rate_delay)
    text = (resp.text or "").strip()
    parsed = try_parse_json(text)
    return parsed if parsed is not None else {"text": text}

def infer_image(image: Image.Image, prompt: str):
    contents = [RESET_PROMPT + prompt.strip(), image]
    resp = client.models.generate_content(model=model_name, contents=contents)
    time.sleep(rate_delay)
    text = (resp.text or "").strip()
    parsed = try_parse_json(text)
    return parsed if parsed is not None else {"text": text}

# ===== 기본 프롬프트(간결 버전) =====
TEXT_PROMPT = (
    "당신은 기사 정보원 분석 전문가입니다. 다음 기사에서 정보원을 추출하고, "
    "정보원별 묘사 프레임을 긍정/중립/부정으로 판정해 JSON으로만 출력하세요.\n"
    '예시: {"sources": ["정보원A","정보원B"], "frames": ["중립","부정"]}'
)

IMAGE_PROMPT = (
    "당신은 보도사진 분석 전문가입니다. 제공된 사진에서 Donald Trump 존재 여부를 판단하고 "
    "감정(emotion: positive/negative/neutral)과 역동성(dynamism: high/medium/low)을 평가해 "
    'JSON으로만 출력하세요. Trump가 없으면 {"trump_present": false}만 반환하세요.'
)

# ===== UI =====
st.title("🤖 멀티모달 LLM 데모 (Gemma 3 27B)")
tab_text, tab_img = st.tabs(["📝 텍스트 분석", "🖼️ 이미지 분석"])

with tab_text:
    st.subheader("기사 텍스트 → 정보원 & 프레임 판정")
    article = st.text_area(
        "기사 본문", height=180,
        value="Donald Trump와 Nancy Pelosi가 회의장에서 격렬히 논쟁을 벌였다..."
    )
    user_prompt = st.text_area("프롬프트(옵션)", value=TEXT_PROMPT, height=120)
    if st.button("텍스트 분석 실행", type="primary", use_container_width=True):
        if not article.strip():
            st.error("기사 본문을 입력해 주세요.")
        else:
            with st.spinner("분석 중..."):
                try:
                    result = infer_text(article, user_prompt or TEXT_PROMPT)
                    st.success("완료")
                    st.json(result)
                    st.download_button("결과 JSON 다운로드", data=json.dumps(result, ensure_ascii=False, indent=2),
                                       file_name="text_result.json", mime="application/json")
                except Exception as e:
                    st.error(f"오류: {e}")

with tab_img:
    st.subheader("보도사진 → 인물 존재·감정·역동성 분석")
    col1, col2 = st.columns(2)
    with col1:
        img_url = st.text_input("이미지 URL")
    with col2:
        file = st.file_uploader("이미지 업로드", type=["jpg","jpeg","png","webp","gif","bmp"])

    img_prompt = st.text_area("프롬프트(옵션)", value=IMAGE_PROMPT, height=120)
    if st.button("이미지 분석 실행", type="primary", use_container_width=True):
        try:
            if file is not None:
                image = Image.open(io.BytesIO(file.read())).convert("RGB")
            elif img_url.strip():
                image = load_image(img_url.strip())
            else:
                st.error("이미지 URL을 입력하거나 파일을 업로드해 주세요.")
                st.stop()

            st.image(image, caption="입력 이미지", use_container_width=True)
            with st.spinner("분석 중..."):
                result = infer_image(image, img_prompt or IMAGE_PROMPT)
            st.success("완료")
            st.json(result)
            st.download_button("결과 JSON 다운로드", data=json.dumps(result, ensure_ascii=False, indent=2),
                               file_name="image_result.json", mime="application/json")
        except Exception as e:
            st.error(f"오류: {e}")
